## Data Visualizations

You can use MoviePy for data visualizations. Here's an example by [Zulko](http://zulko.github.io/blog/2014/11/29/data-animations-with-python-and-moviepy/).

In [2]:
# Thanks Zulko!
# http://zulko.github.io/blog/2014/11/29/data-animations-with-python-and-moviepy/

from moviepy.editor import VideoClip
import numpy as np
from vispy import app, scene
from vispy.gloo.util import _screenshot

canvas = scene.SceneCanvas(keys='interactive')
view = canvas.central_widget.add_view()
view.set_camera('turntable', mode='perspective', up='z', distance=2,
                azimuth=30., elevation=65.)

xx, yy = np.arange(-1,1,.02),np.arange(-1,1,.02)
X,Y = np.meshgrid(xx,yy)
R = np.sqrt(X**2+Y**2)
Z = lambda t : 0.1*np.sin(10*R-2*np.pi*t)
surface = scene.visuals.SurfacePlot(x= xx-0.1, y=yy+0.2, z= Z(0),
                        shading='smooth', color=(0.5, 0.5, 1, 1))
view.add(surface)
canvas.show()

# ANIMATE WITH MOVIEPY

def make_frame(t):
    surface.set_data(z = Z(t)) # Update the mathematical surface
    canvas.on_draw(None) # Update the image on Vispy's canvas
    return _screenshot((0,0,canvas.size[0],canvas.size[1]))[:,:,:3]

animation = VideoClip(make_frame, duration=1).resize(width=350)
animation.write_gif('sinc_vispy.gif', fps=20, opt='OptimizePlus')

INFO: Could not import backend "PyQt4":
No module named PyQt4
INFO: Could not import backend "PyQt5":
No module named PyQt5
INFO: Could not import backend "PySide":
No module named PySide
INFO: Could not import backend "Pyglet":
No module named pyglet
INFO: Could not import backend "Glfw":
GLFW library not found
INFO: Could not import backend "SDL2":
No module named sdl2
INFO: Could not import backend "wx":
No module named wx
INFO: Could not import backend "EGL":
EGL library not found


RuntimeError: Could not import any of the backends. You need to install any of ['PyQt4', 'PyQt5', 'PySide', 'Pyglet', 'Glfw', 'SDL2', 'wx', 'EGL']. We recommend PyQt

## Using PyGame

Sometimes you need somthing more advanced than rendering a mask or text on a video. As long as you can generate an 2D NumPy array of colors, you can anything!

Here's an example using PyGame:

In [ ]:
import os
import pygame
import moviepy.editor as mpy

# Headless SDL
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Start up pygame
pygame.init()

# Going to hide the main display (make it small)
pygame.display.set_mode((1,1))

# Buffer we're going to draw changes in
screen = pygame.Surface((1024, 768), pygame.SRCALPHA, 32)

# Get your own duck
duckface = pygame.image.load("duck.bmp")
position = duckface.get_rect()

# Function to get a frame 
# Returns a NumPy-style array size WIDTHxHEIGHTx3
# Represeting a 2D RGB field. (Each component is 8-bit)

def next_frame(dt):
    global position # globals, lol

    # Translate by (2, 0)
    position = position.move(2, 0)

    # Fill the screen with black
    screen.fill(0x00000000)

    # Draw duckface
    screen.blit(duckface, position)

    # Bring the image from the other buffer to 
    # main
    pygame.display.flip()

    # For some reason, we have to do this or
    # else the image comes out flipped
    newscreen = pygame.transform.rotate(screen, 90)

    # Reverse the columns of the pixel array
    # We could do it with MoviePy with .fx(vfx.mirror_y)
    return pygame.surfarray.array3d(newscreen)[:,::-1]
    

duration = 3 # in seconds
animation = mpy.VideoClip(next_frame, duration=duration)
animation.write_videofile("./video.mp4", fps=60) # 60fps or die

## Dependencies

I've only tested this on macOS. Please update the instructions if you find yourself installing this on other platforms.

### Install VTK

`brew install vtk --with-qt` (Be warned: this takes a while)

In [ ]:
# pip requirements.txt

